In [1]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from transformers import BertTokenizer, BertModel
import torch
import nltk
from nltk.corpus import stopwords
# Naive Bayes 모델
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB

# NLTK 불용어 다운로드 및 로드
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

#gpu 메모리 초기화
def clear_memory():
    torch.cuda.empty_cache()
    torch.cuda.ipc_collect()

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# 데이터 로드
try:
    cve_data = pd.read_csv('allitems_cleaned.csv', encoding='utf-8', low_memory=False)
    attack_data = pd.read_csv('enterprise-attack-v15.1.csv', encoding='utf-8')
    d3fend_data = pd.read_csv('d3fend.csv', encoding='utf-8')
except UnicodeDecodeError:
    cve_data = pd.read_csv('allitems_cleaned.csv', encoding='ISO-8859-1', low_memory=False)
    attack_data = pd.read_csv('enterprise-attack-v15.1.csv', encoding='ISO-8859-1')
    d3fend_data = pd.read_csv('d3fend.csv', encoding='ISO-8859-1')
    

In [3]:

# 텍스트 임베딩 함수
def embed_text(texts, tokenizer, model, device, max_length=512, padding=True, truncation=True):
    all_embeddings = []
    for text in texts:
        inputs = tokenizer(text, return_tensors='pt', truncation=truncation, padding=padding, max_length=max_length).to(device)
        with torch.no_grad():
            outputs = model(**inputs)
        embeddings = outputs.last_hidden_state.mean(dim=1).cpu().numpy()
        all_embeddings.append(embeddings[0])
    return all_embeddings

# 배치 임베딩 함수
def embed_batch(texts, tokenizer, model, device, batch_size=16):
    all_embeddings = []
    for i in range(0, len(texts), batch_size):
        batch_texts = texts[i:i + batch_size]
        inputs = tokenizer(batch_texts, return_tensors='pt', truncation=True, padding=True, max_length=512).to(device)
        with torch.no_grad():
            outputs = model(**inputs)
        embeddings = outputs.last_hidden_state.mean(dim=1).cpu().numpy()
        all_embeddings.extend(embeddings)
    return all_embeddings


In [26]:
# BERT 모델과 토크나이저 로드
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased').to(device)

cve_data['Description'] = cve_data['Description'].astype(str).fillna('')

# CVE 설명 임베딩
cve_data['embedding'] = embed_batch(cve_data['Description'].tolist(), tokenizer, model, device)


In [5]:
cve_data.tail()


,Name,Status,Description,References,Phase,Votes,Comments,embedding
248181,CVE-2024-4309,Candidate,SQL injection vulnerability in HubBank affecti...,MISC:https://www.incibe.es/en/incibe-cert/noti...,Assigned (20240429),None (candidate not yet proposed),NaN,"[-0.23526987, -0.30368063, 0.18257909, 0.09077..."
248182,CVE-2024-4310,Candidate,Cross-site Scripting (XSS) vulnerability in Hu...,MISC:https://www.incibe.es/en/incibe-cert/noti...,Assigned (20240429),None (candidate not yet proposed),NaN,"[-0.14796196, -0.19170775, 0.08765839, 0.10771..."
248183,CVE-2024-4327,Candidate,A vulnerability was found in Apryse WebViewer ...,MISC:Submit #321231 | apryse WebViewe 10.8.0 C...,Assigned (20240429),None (candidate not yet proposed),NaN,"[-0.43377003, -0.11131292, 0.17947115, -0.0620..."
248184,CVE-2024-4336,Candidate,"Adive Framework 2.0.8, does not sufficiently e...",MISC:https://www.incibe.es/en/incibe-cert/noti...,Assigned (20240430),None (candidate not yet proposed),NaN,"[-0.27849382, -0.036491398, 0.016790077, 0.109..."
248185,CVE-2024-4337,Candidate,"Adive Framework 2.0.8, does not sufficiently e...",MISC:https://www.incibe.es/en/incibe-cert/noti...,Assigned (20240430),None (candidate not yet proposed),NaN,"[-0.30299723, 0.026778739, -0.018162878, 0.085..."


아래 사용 안함. 사용하는곳부터 다시 체크

In [27]:
# ATT&CK 기술 설명 임베딩
attack_data['description'] = attack_data['description'].astype(str).fillna('')
attack_data['bert_embedding'] = embed_batch(attack_data['description'].tolist(), tokenizer, model, device)

# D3FEND 기술 설명 임베딩
d3fend_data['Definition'] = d3fend_data['Definition'].astype(str).fillna('')
d3fend_data['bert_embedding'] = embed_batch(d3fend_data['Definition'].tolist(), tokenizer, model, device)

In [28]:
#유사도 계산 및 매칭 함수
def find_best_match(cve_emb, tech_data, embedding_column):
    similarities = cosine_similarity([cve_emb], tech_data[embedding_column].tolist())
    best_match_idx = similarities.argmax()
    return tech_data.iloc[best_match_idx]

# CVE 데이터를 ATT&CK 및 D3FEND와 매칭
cve_data['bert_attack_match'] = [find_best_match(emb, attack_data, 'bert_embedding')['ID'] for emb in cve_data['embedding']]
cve_data['bert_d3fend_match'] = [find_best_match(emb, d3fend_data, 'bert_embedding')['ID'] for emb in cve_data['embedding']]

#bert 끝
clear_memory()


In [4]:


# Naive Bayes 모델

# TF-IDF 벡터화 및 Naive Bayes 모델 학습
tfidf = TfidfVectorizer(stop_words='english')

# 모든 데이터셋을 변환할 때 동일한 TF-IDF 벡터화기를 사용
combined_descriptions = pd.concat([attack_data['description'], d3fend_data['Definition'], cve_data['Description']])
tfidf.fit(combined_descriptions)

# ATT&CK 기술 설명 벡터화 및 모델 학습
attack_tfidf = tfidf.transform(attack_data['description'])
nb_attack_model = MultinomialNB().fit(attack_tfidf, attack_data['ID'])

# D3FEND 기술 설명 벡터화 및 모델 학습
d3fend_tfidf = tfidf.transform(d3fend_data['Definition'])
nb_d3fend_model = MultinomialNB().fit(d3fend_tfidf, d3fend_data['ID'])

# CVE 설명 벡터화
cve_tfidf = tfidf.transform(cve_data['Description']) 

# CVE 데이터를 ATT&CK 및 D3FEND와 매칭
cve_data['nb_attack_match'] = nb_attack_model.predict(cve_tfidf)
cve_data['nb_d3fend_match'] = nb_d3fend_model.predict(cve_tfidf)

#나이브 베이즈 끝
clear_memory()


ValueError: np.nan is an invalid document, expected byte or unicode string.

In [30]:

secbert_tokenizer = BertTokenizer.from_pretrained('SecBERT-main')
secbert_model = BertModel.from_pretrained('SecBERT-main').to(device)

# CVE 설명 임베딩
cve_data['secbert_embedding'] = embed_batch(cve_data['Description'].tolist(), secbert_tokenizer, secbert_model, device)

# ATT&CK 기술 설명 임베딩
attack_data['secbert_embedding'] = embed_batch(attack_data['description'].tolist(), secbert_tokenizer, secbert_model, device)

# D3FEND 기술 설명 임베딩
d3fend_data['secbert_embedding'] = embed_batch(d3fend_data['Definition'].tolist(), secbert_tokenizer, secbert_model, device)

# CVE 데이터를 ATT&CK 및 D3FEND와 매칭
cve_data['secbert_attack_match'] = [find_best_match(emb, attack_data, 'secbert_embedding')['ID'] for emb in cve_data['secbert_embedding']]
cve_data['secbert_d3fend_match'] = [find_best_match(emb, d3fend_data, 'secbert_embedding')['ID'] for emb in cve_data['secbert_embedding']]

#secBert 끝
clear_memory()

In [9]:
cve_data.tail()

,Name,Status,Description,References,Phase,Votes,Comments,embedding,bert_attack_match,bert_d3fend_match,secbert_embedding,secbert_attack_match,secbert_d3fend_match,nb_attack_match,nb_d3fend_match
248181,CVE-2024-4309,Candidate,SQL injection vulnerability in HubBank affecti...,MISC:https://www.incibe.es/en/incibe-cert/noti...,Assigned (20240429),None (candidate not yet proposed),NaN,"[-0.23526987, -0.30368063, 0.18257909, 0.09077...",T1505.001,D3-USICA,"[0.03426267, -0.18423666, 0.48651627, 0.347294...",T1007,D3-DQSA,T1003.002,D3-DQSA
248182,CVE-2024-4310,Candidate,Cross-site Scripting (XSS) vulnerability in Hu...,MISC:https://www.incibe.es/en/incibe-cert/noti...,Assigned (20240429),None (candidate not yet proposed),NaN,"[-0.14796196, -0.19170775, 0.08765839, 0.10771...",T1056.003,D3-USICA,"[-0.32574016, -0.2235146, 0.63809836, 0.423240...",T1216,D3-DST,T1137.003,D3-WSAA
248183,CVE-2024-4327,Candidate,A vulnerability was found in Apryse WebViewer ...,MISC:Submit #321231 | apryse WebViewe 10.8.0 C...,Assigned (20240429),None (candidate not yet proposed),NaN,"[-0.43377003, -0.11131292, 0.17947115, -0.0620...",T1548.004,D3-CP,"[-0.774406, 0.90025723, 0.743753, 0.29327062, ...",T1505.002,D3-SYSVA,T1189,D3-SYSVA
248184,CVE-2024-4336,Candidate,"Adive Framework 2.0.8, does not sufficiently e...",MISC:https://www.incibe.es/en/incibe-cert/noti...,Assigned (20240430),None (candidate not yet proposed),NaN,"[-0.27849382, -0.036491398, 0.016790077, 0.109...",T1053,D3-USICA,"[0.052054394, 0.020478204, 0.17020234, 0.42273...",T1003,D3-ORA,T1563.002,D3-WSAA
248185,CVE-2024-4337,Candidate,"Adive Framework 2.0.8, does not sufficiently e...",MISC:https://www.incibe.es/en/incibe-cert/noti...,Assigned (20240430),None (candidate not yet proposed),NaN,"[-0.30299723, 0.026778739, -0.018162878, 0.085...",T1053,D3-USICA,"[0.054356985, 0.028113225, 0.102777794, 0.4503...",T1003,D3-ORA,T1563.002,D3-WSAA


In [13]:
cve_data.drop(columns='embedding', inplace=True)
cve_data.drop(columns='secbert_embedding', inplace=True)
cve_data.to_csv('cve_with_matches__.csv', index=False)


In [12]:
cve_data.tail()

,Name,Status,Description,References,Phase,Votes,Comments,bert_attack_match,bert_d3fend_match,secbert_embedding,secbert_attack_match,secbert_d3fend_match,nb_attack_match,nb_d3fend_match
248181,CVE-2024-4309,Candidate,SQL injection vulnerability in HubBank affecti...,MISC:https://www.incibe.es/en/incibe-cert/noti...,Assigned (20240429),None (candidate not yet proposed),NaN,T1505.001,D3-USICA,"[0.03426267, -0.18423666, 0.48651627, 0.347294...",T1007,D3-DQSA,T1003.002,D3-DQSA
248182,CVE-2024-4310,Candidate,Cross-site Scripting (XSS) vulnerability in Hu...,MISC:https://www.incibe.es/en/incibe-cert/noti...,Assigned (20240429),None (candidate not yet proposed),NaN,T1056.003,D3-USICA,"[-0.32574016, -0.2235146, 0.63809836, 0.423240...",T1216,D3-DST,T1137.003,D3-WSAA
248183,CVE-2024-4327,Candidate,A vulnerability was found in Apryse WebViewer ...,MISC:Submit #321231 | apryse WebViewe 10.8.0 C...,Assigned (20240429),None (candidate not yet proposed),NaN,T1548.004,D3-CP,"[-0.774406, 0.90025723, 0.743753, 0.29327062, ...",T1505.002,D3-SYSVA,T1189,D3-SYSVA
248184,CVE-2024-4336,Candidate,"Adive Framework 2.0.8, does not sufficiently e...",MISC:https://www.incibe.es/en/incibe-cert/noti...,Assigned (20240430),None (candidate not yet proposed),NaN,T1053,D3-USICA,"[0.052054394, 0.020478204, 0.17020234, 0.42273...",T1003,D3-ORA,T1563.002,D3-WSAA
248185,CVE-2024-4337,Candidate,"Adive Framework 2.0.8, does not sufficiently e...",MISC:https://www.incibe.es/en/incibe-cert/noti...,Assigned (20240430),None (candidate not yet proposed),NaN,T1053,D3-USICA,"[0.054356985, 0.028113225, 0.102777794, 0.4503...",T1003,D3-ORA,T1563.002,D3-WSAA


In [31]:
#매칭 근거 확인
def find_key_tokens(cve_text, tech_text, tokenizer, model, device):
    cve_embeds, cve_inputs = embed_text(cve_text, tokenizer, model, device)
    tech_embeds, tech_inputs = embed_text(tech_text, tokenizer, model, device)

    cve_tokens = tokenizer.convert_ids_to_tokens(cve_inputs['input_ids'][0])
    tech_tokens = tokenizer.convert_ids_to_tokens(tech_inputs['input_ids'][0])

    cve_embeds = cve_embeds.squeeze(0)  # Remove batch dimension
    tech_embeds = tech_embeds.squeeze(0)  # Remove batch dimension

    similarities = cosine_similarity(cve_embeds.cpu().numpy(), tech_embeds.cpu().numpy())

    cve_key_tokens = []
    tech_key_tokens = []

    for i in range(len(similarities)):
        cve_token = cve_tokens[i]
        tech_token = tech_tokens[similarities[i].argmax()]
        cve_key_tokens.append((cve_token, similarities[i].max()))
        tech_key_tokens.append((tech_token, similarities[i].max()))

    return cve_key_tokens, tech_key_tokens

다음부터 다시 시작

In [4]:
# 데이터 전처리: 모든 설명을 문자열로 변환하고, NaN 값을 빈 문자열로 대체
cve_data['Description'] = cve_data['Description'].astype(str).fillna('')
attack_data['description'] = attack_data['description'].astype(str).fillna('')
d3fend_data['Definition'] = d3fend_data['Definition'].astype(str).fillna('')

# BERT 모델과 토크나이저 로드
bert_tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
bert_model = BertModel.from_pretrained('bert-base-uncased').to(device)

# secBERT 모델과 토크나이저 로드
secbert_tokenizer = BertTokenizer.from_pretrained('SecBERT-main')
secbert_model = BertModel.from_pretrained('SecBERT-main').to(device)

C:\Users\user\OneDrive - Sejong University\matching_cve\lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [5]:
#clear_memory()

# 주요 단어 찾기 함수 (불용어 제거 포함)
def find_key_tokens(cve_text, tech_text, tokenizer, model, device, max_length=512, padding=True, truncation=True):
    cve_embeds = embed_text([cve_text], tokenizer, model, device, max_length, padding, truncation)
    tech_embeds = embed_text([tech_text], tokenizer, model, device, max_length, padding, truncation)

    cve_inputs = tokenizer(cve_text, return_tensors='pt', truncation=truncation, padding=padding, max_length=max_length)
    tech_inputs = tokenizer(tech_text, return_tensors='pt', truncation=truncation, padding=padding, max_length=max_length)

    cve_tokens = tokenizer.convert_ids_to_tokens(cve_inputs['input_ids'][0])
    tech_tokens = tokenizer.convert_ids_to_tokens(tech_inputs['input_ids'][0])

    cve_embeds = torch.tensor(cve_embeds).squeeze(0)  # Remove batch dimension
    tech_embeds = torch.tensor(tech_embeds).squeeze(0)  # Remove batch dimension

    cve_embeds = cve_embeds.reshape(1, -1)  # Convert to 2D array
    tech_embeds = tech_embeds.reshape(1, -1)  # Convert to 2D array

    similarities = cosine_similarity(cve_embeds, tech_embeds)

    cve_key_tokens = []
    tech_key_tokens = []

    for i in range(len(similarities)):
        cve_token = cve_tokens[i]
        tech_token = tech_tokens[similarities[i].argmax()]
        if cve_token not in stop_words and tech_token not in stop_words and cve_token != '[CLS]' and cve_token != '[SEP]' and tech_token != '[CLS]' and tech_token != '[SEP]':
            cve_key_tokens.append((cve_token, similarities[i].max()))
            tech_key_tokens.append((tech_token, similarities[i].max()))

    return cve_key_tokens, tech_key_tokens

In [6]:
# Naive Bayes 주요 단어 찾기 함수
def find_key_tokens_nb(cve_text, tech_text, tfidf):
    cve_tfidf = tfidf.transform([cve_text])
    tech_tfidf = tfidf.transform([tech_text])

    cve_tokens = tfidf.get_feature_names_out()
    tech_tokens = tfidf.get_feature_names_out()

    cve_key_tokens = []
    tech_key_tokens = []

    cve_array = cve_tfidf.toarray().flatten()
    tech_array = tech_tfidf.toarray().flatten()


    for i in range(len(cve_array)):
        if cve_array[i] > 0 and cve_tokens[i] not in stop_words and tech_tokens[tech_array.argmax()] not in stop_words:
            cve_token = cve_tokens[i]
            tech_token = tech_tokens[tech_array.argmax()]
            cve_key_tokens.append((cve_token, cve_array[i]))
            tech_key_tokens.append((tech_token, tech_array[tech_array.argmax()]))

    return cve_key_tokens, tech_key_tokens


In [7]:
# 모든 텍스트 데이터를 합침
all_descriptions = pd.concat([cve_data['Description'], attack_data['description'], d3fend_data['Definition']])

# Naive Bayes 모델
tfidf = TfidfVectorizer(stop_words='english')

# 모든 텍스트 데이터를 사용하여 TF-IDF 학습
tfidf.fit(all_descriptions)

# ATT&CK 및 D3FEND 기술 설명 벡터화 및 모델 학습
attack_tfidf = tfidf.transform(attack_data['description'])
d3fend_tfidf = tfidf.transform(d3fend_data['Definition'])

nb_attack_model = MultinomialNB().fit(attack_tfidf, attack_data['ID'])
nb_d3fend_model = MultinomialNB().fit(d3fend_tfidf, d3fend_data['ID'])

# CVE 설명 벡터화
cve_tfidf = tfidf.transform(cve_data['Description'])

# Naive Bayes 매칭 수행
cve_data['nb_attack_match'] = nb_attack_model.predict(cve_tfidf)
cve_data['nb_d3fend_match'] = nb_d3fend_model.predict(cve_tfidf)

In [ ]:
cve_data.tail()

In [8]:
# 유사도 계산 및 매칭 함수 (유사도 점수 포함)
def find_best_match_with_score(cve_emb, tech_data, embedding_column):
    similarities = cosine_similarity([cve_emb], tech_data[embedding_column].tolist())
    best_match_idx = similarities.argmax()
    return tech_data.iloc[best_match_idx], similarities[0, best_match_idx]

# CVE 설명 임베딩 (전체 데이터)
cve_data['bert_embedding'] = embed_text(cve_data['Description'].tolist(), bert_tokenizer, bert_model, device)
cve_data['secbert_embedding'] = embed_text(cve_data['Description'].tolist(), secbert_tokenizer, secbert_model, device)

In [10]:

# ATT&CK 및 D3FEND 기술 설명 임베딩
attack_data['bert_embedding'] = embed_text(attack_data['description'].tolist(), bert_tokenizer, bert_model, device)
d3fend_data['bert_embedding'] = embed_text(d3fend_data['Definition'].tolist(), bert_tokenizer, bert_model, device)
attack_data['secbert_embedding'] = embed_text(attack_data['description'].tolist(), secbert_tokenizer, secbert_model, device)
d3fend_data['secbert_embedding'] = embed_text(d3fend_data['Definition'].tolist(), secbert_tokenizer, secbert_model, device)


In [ ]:
# 매칭 근거를 포함한 데이터프레임 생성
matching_data = []

for idx, row in cve_data.iterrows():
    cve_text = row['Description']

    # BERT 매칭
    bert_attack_match, bert_attack_score = find_best_match_with_score(row['bert_embedding'], attack_data, 'bert_embedding')
    bert_d3fend_match, bert_d3fend_score = find_best_match_with_score(row['bert_embedding'], d3fend_data, 'bert_embedding')
    
    attack_match_bert_text = attack_data[attack_data['ID'] == bert_attack_match['ID']].iloc[0]['description']
    d3fend_match_bert_text = d3fend_data[d3fend_data['ID'] == bert_d3fend_match['ID']].iloc[0]['Definition']
    
    cve_key_tokens_bert, attack_key_tokens_bert = find_key_tokens(cve_text, attack_match_bert_text, bert_tokenizer, bert_model, device)
    _, d3fend_key_tokens_bert = find_key_tokens(cve_text, d3fend_match_bert_text, bert_tokenizer, bert_model, device)

    # secBERT 매칭
    secbert_attack_match, secbert_attack_score = find_best_match_with_score(row['secbert_embedding'], attack_data, 'secbert_embedding')
    secbert_d3fend_match, secbert_d3fend_score = find_best_match_with_score(row['secbert_embedding'], d3fend_data, 'secbert_embedding')

    attack_match_secbert_text = attack_data[attack_data['ID'] == secbert_attack_match['ID']].iloc[0]['description']
    d3fend_match_secbert_text = d3fend_data[d3fend_data['ID'] == secbert_d3fend_match['ID']].iloc[0]['Definition']

    cve_key_tokens_secbert, attack_key_tokens_secbert = find_key_tokens(cve_text, attack_match_secbert_text, secbert_tokenizer, secbert_model, device)
    _, d3fend_key_tokens_secbert = find_key_tokens(cve_text, d3fend_match_secbert_text, secbert_tokenizer, secbert_model, device)

    # Naive Bayes 매칭
    attack_match_nb = attack_data[attack_data['ID'] == row['nb_attack_match']].iloc[0]['description']
    d3fend_match_nb = d3fend_data[d3fend_data['ID'] == row['nb_d3fend_match']].iloc[0]['Definition']

    cve_key_tokens_nb, attack_key_tokens_nb = find_key_tokens_nb(cve_text, attack_match_nb, tfidf)
    _, d3fend_key_tokens_nb = find_key_tokens_nb(cve_text, d3fend_match_nb, tfidf)


    matching_data.append({
        'cve_id': row['Name'],
        'cve_description': cve_text,
        'bert_attack_match': bert_attack_match['ID'],
        'bert_d3fend_match': bert_d3fend_match['ID'],
        'secbert_attack_match': secbert_attack_match['ID'],
        'secbert_d3fend_match': secbert_d3fend_match['ID'],
        'nb_attack_match': row['nb_attack_match'],
        'nb_d3fend_match': row['nb_d3fend_match'],
        'cve_key_tokens_bert': cve_key_tokens_bert,
        'attack_key_tokens_bert': attack_key_tokens_bert,
        'd3fend_key_tokens_bert': d3fend_key_tokens_bert,
        'cve_key_tokens_secbert': cve_key_tokens_secbert,
        'attack_key_tokens_secbert': attack_key_tokens_secbert,
        'd3fend_key_tokens_secbert': d3fend_key_tokens_secbert,
        'cve_key_tokens_nb': cve_key_tokens_nb,
        'attack_key_tokens_nb': attack_key_tokens_nb,
        'd3fend_key_tokens_nb': d3fend_key_tokens_nb
    })



In [17]:
matching_data

[{'cve_id': 'CVE-2024-4337',
  'cve_description': 'Adive Framework 2.0.8, does not sufficiently encode user-controlled inputs, resulting in a persistent Cross-Site Scripting (XSS) vulnerability via the /adive/admin/nav/add, in multiple parameters. This vulnerability allows an attacker to retrieve the session details of an authenticated user.',
  'bert_attack_match': 'T1216',
  'bert_d3fend_match': 'D3-CP',
  'secbert_attack_match': 'T1204.001',
  'secbert_d3fend_match': 'D3-NVA',
  'nb_attack_match': 'T1563.002',
  'nb_d3fend_match': 'D3-WSAA',
  'cve_key_tokens_bert': [],
  'attack_key_tokens_bert': [],
  'd3fend_key_tokens_bert': [],
  'cve_key_tokens_secbert': [],
  'attack_key_tokens_secbert': [],
  'd3fend_key_tokens_secbert': [],
  'cve_key_tokens_nb': [('add', 0.1643408671295955),
   ('adive', 0.6461478751506837),
   ('admin', 0.12153632227517967),
   ('allows', 0.05344485774671441),
   ('attacker', 0.07992927057017242),
   ('authenticated', 0.1032606916013725),
   ('controlled'

In [42]:
import xml.etree.ElementTree as ET
# XML 생성 함수
def create_xml(data):
    root = ET.Element("Root")

    for item in data:
        cve_entry = ET.SubElement(root, "CVE_Entry")

        for key, value in item.items():
            if isinstance(value, list) and all(isinstance(i, tuple) for i in value):
                tokens_element = ET.SubElement(cve_entry, key)
                for token, score in value:
                    token_element = ET.SubElement(tokens_element, "Token")
                    token_element.set("score", str(score))
                    token_element.text = token
            else:
                element = ET.SubElement(cve_entry, key)
                element.text = str(value)

    tree = ET.ElementTree(root)
    tree.write("output_after.xml", encoding="utf-8", xml_declaration=True)

create_xml(matching_data)

---------------------------
토큰화된 단어의 상관관계성 분석
단어 토큰화된것 출력해보기
---------------------------

In [27]:

# 평가 함수 참조 데이터가 없기에 불가능.
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score

def evaluate(true_labels, predicted_labels):
    accuracy = accuracy_score(true_labels, predicted_labels)
    f1 = f1_score(true_labels, predicted_labels, average='weighted')
    precision = precision_score(true_labels, predicted_labels, average='weighted')
    recall = recall_score(true_labels, predicted_labels, average='weighted')
    return accuracy, f1, precision, recall

# 참조 데이터(ground truth labels)
true_attack_matches = cve_data['true_attack_id'].tolist()
true_d3fend_matches = cve_data['true_d3fend_id'].tolist()

# BERT 모델 평가
bert_attack_accuracy, bert_attack_f1, bert_attack_precision, bert_attack_recall = evaluate(true_attack_matches, cve_data['bert_attack_match'])
bert_d3fend_accuracy, bert_d3fend_f1, bert_d3fend_precision, bert_d3fend_recall = evaluate(true_d3fend_matches, cve_data['bert_d3fend_match'])

print(f'BERT - ATT&CK 매칭: 정확도={bert_attack_accuracy}, F1-스코어={bert_attack_f1}, 정밀도={bert_attack_precision}, 재현율={bert_attack_recall}')
print(f'BERT - D3FEND 매칭: 정확도={bert_d3fend_accuracy}, F1-스코어={bert_d3fend_f1}, 정밀도={bert_d3fend_precision}, 재현율={bert_d3fend_recall}')

# Naive Bayes 모델 평가
nb_attack_accuracy, nb_attack_f1, nb_attack_precision, nb_attack_recall = evaluate(true_attack_matches, cve_data['nb_attack_match'])
nb_d3fend_accuracy, nb_d3fend_f1, nb_d3fend_precision, nb_d3fend_recall = evaluate(true_d3fend_matches, cve_data['nb_d3fend_match'])

print(f'Naive Bayes - ATT&CK 매칭: 정확도={nb_attack_accuracy}, F1-스코어={nb_attack_f1}, 정밀도={nb_attack_precision}, 재현율={nb_attack_recall}')
print(f'Naive Bayes - D3FEND 매칭: 정확도={nb_d3fend_accuracy}, F1-스코어={nb_d3fend_f1}, 정밀도={nb_d3fend_precision}, 재현율={nb_d3fend_recall}')
# 
# # secBERT 모델 평가
# secbert_attack_accuracy, secbert_attack_f1, secbert_attack_precision, secbert_attack_recall = evaluate(true_attack_matches, cve_data['secbert_attack_match'])
# secbert_d3fend_accuracy, secbert_d3fend_f1, secbert_d3fend_precision, secbert_d3fend_recall = evaluate(true_d3fend_matches, cve_data['secbert_d3fend_match'])
# 
# print(f'secBERT - ATT&CK 매칭: 정확도={secbert_attack_accuracy}, F1-스코어={secbert_attack_f1}, 정밀도={secbert_attack_precision}, 재현율={secbert_attack_recall}')
# print(f'secBERT - D3FEND 매칭: 정확도={secbert_d3fend_accuracy}, F1-스코어={secbert_d3fend_f1}, 정밀도={secbert_d3fend_precision}, 재현율={secbert_d3fend_recall}')


KeyError: 'true_attack_id'

In [13]:
cve_data.tail()

,Name,Status,Description,References,Phase,Votes,Comments,embedding,bert_attack_match,bert_d3fend_match,nb_attack_match,nb_d3fend_match
248181,CVE-2024-4309,Candidate,SQL injection vulnerability in HubBank affecti...,MISC:https://www.incibe.es/en/incibe-cert/noti...,Assigned (20240429),None (candidate not yet proposed),NaN,"[-0.23526987, -0.30368063, 0.18257909, 0.09077...",T1505.001,D3-USICA,T1003.002,D3-DQSA
248182,CVE-2024-4310,Candidate,Cross-site Scripting (XSS) vulnerability in Hu...,MISC:https://www.incibe.es/en/incibe-cert/noti...,Assigned (20240429),None (candidate not yet proposed),NaN,"[-0.14796196, -0.19170775, 0.08765839, 0.10771...",T1056.003,D3-USICA,T1137.003,D3-WSAA
248183,CVE-2024-4327,Candidate,A vulnerability was found in Apryse WebViewer ...,MISC:Submit #321231 | apryse WebViewe 10.8.0 C...,Assigned (20240429),None (candidate not yet proposed),NaN,"[-0.43377003, -0.11131292, 0.17947115, -0.0620...",T1548.004,D3-CP,T1189,D3-SYSVA
248184,CVE-2024-4336,Candidate,"Adive Framework 2.0.8, does not sufficiently e...",MISC:https://www.incibe.es/en/incibe-cert/noti...,Assigned (20240430),None (candidate not yet proposed),NaN,"[-0.27849382, -0.036491398, 0.016790077, 0.109...",T1053,D3-USICA,T1563.002,D3-WSAA
248185,CVE-2024-4337,Candidate,"Adive Framework 2.0.8, does not sufficiently e...",MISC:https://www.incibe.es/en/incibe-cert/noti...,Assigned (20240430),None (candidate not yet proposed),NaN,"[-0.30299723, 0.026778739, -0.018162878, 0.085...",T1053,D3-USICA,T1563.002,D3-WSAA


In [1]:
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cpu


In [1]:
import torch

print(torch.cuda.is_available())  # True가 출력되어야 합니다.
print(torch.cuda.current_device())
print(torch.cuda.get_device_name(torch.cuda.current_device()))


True
0
NVIDIA GeForce RTX 4090
